In [113]:
import os
import fitz
from tqdm import tqdm

In [114]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [121]:
zip_path = '兆豐'
count_true = 0
possible_rate = ['逢低買進', '區間操作', '買進']
check_source = '本刊所刊載之內容僅做為參考，惟已力求正確與完整'
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)
            rect = page.rect
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            if check_source in text_check_source :
                # 檢查版本                
                clip_check_new_report = fitz.Rect(rect.width/2, 0, rect.width, 150)
                text_check_new_report = page.get_text(clip=clip_check_new_report)
                if '訪談速報' in text_check_new_report or '個股報告' in text_check_new_report:
                    # 提取評價的第一種方法
                    try:
                        clip_old_report_1 = fitz.Rect(400, 0, rect.width, 200)
                        text_old_report_1 = page.get_text(clip=clip_old_report_1).strip()
                        text_old_report_1 = text_old_report_1.split('目標價')[1].strip()
                        rate_1 = text_old_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                        # 提取評價的第二種方法
                    try:
                        clip_old_report_2 = fitz.Rect(410, 140, 490, 165)
                        rate_2 = page.get_text(clip=clip_old_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                elif '晨會報告' in text_check_new_report :
                    rate = 'wrong'
                else :
                    print(f'{text_check_new_report}, {directory_path} NULL')
            else:
                # False
                print(f'{directory_path} is not posted by 兆豐投顧')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print(directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print(directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

100%|██████████| 521/521 [00:29<00:00, 17.81it/s]

共521件, 成功521件, 失敗0件, 成功率:100.0%


In [116]:
import fitz
doc = fitz.open(r'兆豐\1101_台泥_2019-03-12_兆豐_20190312-訪談速報-台泥(1101).pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [117]:
# 檢查是否由統一投顧出版
page_check_source = doc.load_page(-1)
text_check_source = page_check_source.get_text()
if check_source in text_check_source :
    print('True')

True


In [118]:
# 檢查版本
clip_check_new_report = fitz.Rect(rect.width/2, 0, rect.width, 150)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_new_report)
pix.save("page-clip_check_new_report.png")
text_check_new_report = page.get_text(clip=clip_check_new_report)
if '訪談速報' in text_check_new_report :
    print('訪談速報')
elif '個股報告' in text_check_new_report :
    print('個股報告')
# elif '初次報告' in text_check_new_report :
#     print('初次報告')
# elif '投資早報' in text_check_new_report or '港股投資' in text_check_new_report:
#     print('投資早報 or 港股投資')
else :
    print('NULL')


訪談速報


In [119]:
# 報告為舊版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(400, 0, rect.width, 200)
    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_old_report_1.png")
    text_old_report_1 = text_old_report_1.split('目標價')[1].strip()
    rate_1 = text_old_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

'逢低買進'

In [120]:
# 報告為舊版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(410, 140, 490, 165)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_old_report_2.png")
except:
    rate_2 = None
rate_2

'逢低買進'